In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as T
import time

# FeatUp utilities
from featup.util import norm, unnorm
from featup.plotting import plot_feats

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances
import torch.nn.functional as F
from scipy.ndimage import gaussian_filter
from scipy.stats import median_abs_deviation

# Anomaly region detection and visualization
from skimage import measure
import matplotlib.patches as patches

# SAM2 imports
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
import cv2

# PCA for manual visualization
from sklearn.decomposition import PCA


# Set the CUDA device to GPU 4
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# --- Configuración ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 224 # DINOv2 input size
BACKBONE_PATCH_SIZE = 14 # DINOv2 ViT-S/14 patch size
use_norm = True

H_prime = input_size // BACKBONE_PATCH_SIZE
W_prime = input_size // BACKBONE_PATCH_SIZE

# Directorios
TRAIN_GOOD_DIR = '/home/imercatoma/FeatUp/datasets/mvtec_anomaly_detection/hazelnut/train/good'
PLOT_SAVE_ROOT_DIR = '/home/imercatoma/FeatUp/plots_single_3_1/cut_001'
# --- Imagen de Consulta ---
query_image_path = '/home/imercatoma/FeatUp/datasets/mvtec_anomaly_detection/hazelnut/test/cut/001.png'
os.makedirs(PLOT_SAVE_ROOT_DIR, exist_ok=True)

HEATMAPS_SAVE_DIR = os.path.join(PLOT_SAVE_ROOT_DIR, 'individual_heatmaps')
os.makedirs(HEATMAPS_SAVE_DIR, exist_ok=True)

ANOMALY_REGIONS_SAVE_DIR = os.path.join(PLOT_SAVE_ROOT_DIR, 'detected_anomaly_regions')
os.makedirs(ANOMALY_REGIONS_SAVE_DIR, exist_ok=True)

FEATUP_PLOTS_DIR = os.path.join(PLOT_SAVE_ROOT_DIR, 'featup_feature_plots')
os.makedirs(FEATUP_PLOTS_DIR, exist_ok=True)

# Coreset file paths
core_bank_filenames_file = os.path.join(TRAIN_GOOD_DIR, 'core_bank_filenames.pt')
coreset_relevant_flat_features_bank_file = os.path.join(TRAIN_GOOD_DIR, 'coreset_relevant_flat_features_bank.pt')
template_features_bank_coreset_file = os.path.join(TRAIN_GOOD_DIR, 'template_features_bank_coreset.pt')

# --- Cargar Datos del Coreset ---
print("Cargando datos del coreset...")
try:
    coreset_relevant_filenames = torch.load(core_bank_filenames_file)
    coreset_relevant_flat_features_bank = torch.load(coreset_relevant_flat_features_bank_file).to(device)
    coreset_features = torch.load(template_features_bank_coreset_file).to(device)
    print(f"Coreset cargado. Dimensión: {coreset_features.shape}")
except Exception as e:
    print(f"ERROR al cargar archivos del coreset: {e}. Asegúrate de que la Etapa 1 se ejecutó.")
    exit()

# Mover coreset a CPU para sklearn's NearestNeighbors
coreset_features_cpu = coreset_features.cpu().numpy()
# se calcula la distancia coseno == 1 - similitud coseno [0,1] 0 identico, 1 completamente diferente
nn_finder = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine').fit(coreset_features_cpu)
print("NearestNeighbors finder inicializado.")

# --- Cargar Modelo DINOv2 ---
print("Cargando modelo DINOv2...")
#featup_local_path = "/home/imercatoma/FeatUp"
#upsampler = torch.hub.load("mhamilton723/FeatUp", 'dinov2', use_norm=use_norm, source='local').to(device)
#upsampler = torch.hub.load("mhamilton723/FeatUp", 'dinov2', use_norm=use_norm).to(device)
featup_local_path = "/home/imercatoma/FeatUp"
upsampler = torch.hub.load(featup_local_path, 'dinov2', use_norm=use_norm, source='local').to(device)

dinov2_model = upsampler.model
dinov2_model.eval()
print("Modelo DINOv2 cargado.")

# --- Transformación de Imagen ---
transform = T.Compose([
    T.Resize(input_size),
    T.CenterCrop((input_size, input_size)),
    T.ToTensor(),
    norm
])

# --- Carga del Modelo SAM2 (Ámbito Global) ---
checkpoint = "/home/imercatoma/sam2_repo_independent/checkpoints/sam2.1_hiera_small.pt"
model_cfg_name = "configs/sam2.1/sam2.1_hiera_s.yaml"
sam2_model = None

# --- Cargar Modelo SAM2 ---
print(f"Cargando modelo SAM2 desde {checkpoint}...")
sam2_model = build_sam2(model_cfg_name, checkpoint, device=device, apply_postprocessing=True)
sam2_model.eval()
print("Modelo SAM2 cargado.")


# ###########inicio DInoAnomaly adaptado #######

# # --- Función Principal para Puntuaciones de Anomalía ---
# def get_anomaly_scores_for_image(image_path, model, image_transform, nn_finder_instance, H_prime, W_prime, device):
#     try:
#         query_img_pil = Image.open(image_path).convert("RGB")
#         input_tensor = image_transform(query_img_pil).unsqueeze(0).to(device)
#     except Exception as e:
#         print(f"Error cargando/transformando imagen {os.path.basename(image_path)}: {e}")
#         return None, None, None, None, None

#     with torch.no_grad():
#         features_lr = model(input_tensor)
        
#     print("Shape de features_lr:", features_lr.shape)
#     query_patches_flat = features_lr.squeeze(0).permute(1, 2, 0).reshape(-1, features_lr.shape[1]) # shape (H_prime * W_prime, C)
#     print("Primeros 5 patches_flat:")
#     print(query_patches_flat[:5])
#     query_patches_flat_cpu = query_patches_flat.cpu().numpy()
    
#     distances_to_nn, _ = nn_finder_instance.kneighbors(query_patches_flat_cpu)
    
#     print("Shape de distances_to_nn:", distances_to_nn.shape)
#     print("Distancias:", distances_to_nn[5:].flatten())
#     print("Índices:", np.argsort(distances_to_nn, axis=0)[:5].flatten())
#     print("Máximo de distances_to_nn:", np.max(distances_to_nn))
#     print("Mínimo de distances_to_nn:", np.min(distances_to_nn))
    
    
#     patch_anomaly_scores = distances_to_nn.flatten()
#     print("Primeros 5 patch_anomaly_scores:", patch_anomaly_scores[:5])
#     sorted_patch_anomaly_scores = np.sort(patch_anomaly_scores)[::-1]
#     print("Primeros 5 sorted_patch_anomaly_scores:", sorted_patch_anomaly_scores[:20])
#     print("Máximo de sorted_patch_anomaly_scores:", np.max(sorted_patch_anomaly_scores))
#     print("Mínimo de sorted_patch_anomaly_scores:", np.min(sorted_patch_anomaly_scores))
    
#     anomaly_map_lr = patch_anomaly_scores.reshape(H_prime, W_prime)
#     anomaly_map_lr_tensor = torch.from_numpy(anomaly_map_lr).unsqueeze(0).unsqueeze(0).to(device)
#     anomaly_map_upsampled = F.interpolate(anomaly_map_lr_tensor, size=(input_size, input_size), mode='bilinear', align_corners=False)
#     print("Shape de anomaly_map_upsampled:", anomaly_map_upsampled.shape)
#     anomaly_map_upsampled = anomaly_map_upsampled.squeeze().cpu().numpy()
#     print("Primeros 5 valores de anomaly_map_upsampled (aplanado):")
#     print(anomaly_map_upsampled.flatten()[:5])
#     print("Máximo de anomaly_map_upsampled:", np.max(anomaly_map_upsampled))
#     print("Mínimo de anomaly_map_upsampled:", np.min(anomaly_map_upsampled))
    
#     anomaly_map_smoothed = gaussian_filter(anomaly_map_upsampled, sigma=4.0)

#     if anomaly_map_smoothed.max() == anomaly_map_smoothed.min():
#         anomaly_map_final = np.zeros_like(anomaly_map_smoothed, dtype=float)
#     else:
#         anomaly_map_final = (anomaly_map_smoothed - anomaly_map_smoothed.min()) / (anomaly_map_smoothed.max() - anomaly_map_smoothed.min() + 1e-8)

#     return patch_anomaly_scores, sorted_patch_anomaly_scores, query_img_pil, anomaly_map_final, features_lr

# # --- Funciones de Métricas ---
# def calculate_rms(data):
#     return np.sqrt(np.mean(data**2))

# def calculate_mad(data):
#     return median_abs_deviation(data)

# def calculate_median(data):
#     return np.median(data)

# def calculate_quartile(data, q=25):
#     return np.percentile(data, q)

# # --- Funciones de Filtrado de Anomalías ---
# def calculate_spatial_variance_of_top_patches(patch_anomaly_scores, top_percentage=5.5):
#     if patch_anomaly_scores is None or patch_anomaly_scores.size == 0:
#         return np.nan

#     num_patches = patch_anomaly_scores.size
#     num_top = max(1, int(num_patches * top_percentage / 100))
#     top_patch_indices = np.argsort(patch_anomaly_scores)[-num_top:]
#     row_coords = top_patch_indices // W_prime
#     col_coords = top_patch_indices % W_prime

#     std_rows = np.std(row_coords) if len(row_coords) > 1 else 0.0
#     std_cols = np.std(col_coords) if len(col_coords) > 1 else 0.0
#     return std_rows + std_cols

# def calculate_active_patches_count_relative_threshold(patch_anomaly_scores, relative_threshold_percentage):
#     if patch_anomaly_scores is None or patch_anomaly_scores.size == 0: return 0
#     max_val_in_image = np.max(patch_anomaly_scores)
#     if max_val_in_image == 0: return 0
#     threshold_val = max_val_in_image * relative_threshold_percentage
#     return len(patch_anomaly_scores[patch_anomaly_scores > threshold_val])

# def calculate_top_percent_average_anomaly(patch_anomaly_scores, top_percent=1):
#     if patch_anomaly_scores is None or patch_anomaly_scores.size == 0: return 0.0
#     num_patches = patch_anomaly_scores.size
#     num_top = max(1, int(num_patches * top_percent / 100))
#     sorted_scores = np.sort(patch_anomaly_scores)[::-1]
#     return np.mean(sorted_scores[:num_top])

# # --- Generar y Guardar Mapas de Calor ---
# def generate_and_save_heatmap(image_original_pil, anomaly_map_final, sorted_patch_anomaly_scores, save_path, image_name_for_title):
#     num_patches = len(sorted_patch_anomaly_scores)
#     num_top_for_q_score = max(1, int(num_patches * 0.01))
#     q_score = np.mean(sorted_patch_anomaly_scores[:num_top_for_q_score])
#     anomalia_estructural = q_score > 0.27

#     print(f"Q-score: {q_score:.4f}. Anomalía estructural: {'Sí' if anomalia_estructural else 'No'}")

#     plt.figure(figsize=(12, 6))
#     plt.subplot(1, 2, 1)
#     plt.imshow(image_original_pil)
#     plt.title(f'Imagen Original: {image_name_for_title}')
#     plt.axis('off')

#     plt.subplot(1, 2, 2)
#     plt.imshow(anomaly_map_final, cmap='jet')
#     plt.title(f'Mapa de Anomalía (Q-score: {q_score:.2f})')
#     plt.colorbar(label='Puntuación de Anomalía Normalizada')
#     plt.axis('off')

#     plt.tight_layout()
#     plt.savefig(save_path)
#     print(f"Mapa de calor de anomalías guardado en: {save_path}")
#     plt.close()
#     return anomaly_map_final, q_score, anomalia_estructural

base_image_name = os.path.basename(query_image_path)
print(f"\n--- Procesando imagen: {base_image_name} ---")

# RELATIVE_ACTIVE_PATCH_THRESHOLD_PERCENTAGE = 0.80

#current_patch_anomaly_scores, current_sorted_patch_anomaly_scores, query_img_pil, anomaly_map_final_for_regions, query_lr_features = get_anomaly_scores_for_image(
#     query_image_path, dinov2_model, transform, nn_finder, H_prime, W_prime, device
# )


################################

query_img_pil = Image.open(query_image_path).convert("RGB")
input_tensor = transform(query_img_pil).unsqueeze(0).to(device)

with torch.no_grad():
    features_lr = dinov2_model(input_tensor)
    
query_lr_features = features_lr
################################



# print("Variables obtenidas:")
# # Imprimir los 5 primeros valores de los términos solicitados
# if current_patch_anomaly_scores is not None:
#     print("Primeros 5 valores de current_patch_anomaly_scores:")
#     print(current_patch_anomaly_scores[:5])

# if current_sorted_patch_anomaly_scores is not None:
#     print("Primeros 5 valores de current_sorted_patch_anomaly_scores:")
#     print(current_sorted_patch_anomaly_scores[:5])

# if query_img_pil is not None:
#     print("Shape de query_img_pil:")
#     print(query_img_pil.size)  # PIL images use (width, height)

# if anomaly_map_final_for_regions is not None:
#     print("Primeros 5 valores de anomaly_map_final_for_regions (aplanado):")
#     print(anomaly_map_final_for_regions.flatten()[:5])

# if query_lr_features is not None:
#     print("Primeras 5 características de query_lr_features:")
#     print(query_lr_features.reshape(-1, query_lr_features.shape[-1])[:5])

# # Mostrar las 5 primeras características de query_lr_features
# print("Primeras 5 características de query_lr_features:")
# print(query_lr_features.reshape(-1, query_lr_features.shape[-1])[:5])
# print("Máximo de query_lr_features:", torch.max(query_lr_features).item())
# print("Mínimo de query_lr_features:", torch.min(query_lr_features).item())






# # Generar y guardar heatmap
# heatmap_filename = f"heat_{base_image_name}"
# individual_heatmap_save_path = os.path.join(HEATMAPS_SAVE_DIR, heatmap_filename)
# current_anomaly_map_final, current_q_score, current_anomalia_estructural = \
#     generate_and_save_heatmap(query_img_pil, anomaly_map_final_for_regions,
#                                 current_sorted_patch_anomaly_scores,
#                                 individual_heatmap_save_path, base_image_name.replace(".png", ""))

# # Calcular métricas para la imagen actual
# min_val = np.min(current_sorted_patch_anomaly_scores)
# max_val = np.max(current_sorted_patch_anomaly_scores)
# normalized_data = (current_sorted_patch_anomaly_scores - min_val) / (max_val - min_val + 1e-8) if max_val != min_val else np.zeros_like(current_sorted_patch_anomaly_scores)

# A_rms = calculate_rms(normalized_data)
# B_mad = calculate_mad(normalized_data)
# C_median = calculate_median(normalized_data)
# D_q1_normalized = calculate_quartile(normalized_data, q=25)

# dist_rms_mad = A_rms - B_mad
# dist_rms_median = A_rms - C_median
# dist_rms_q1 = A_rms - D_q1_normalized
# spatial_var = calculate_spatial_variance_of_top_patches(current_patch_anomaly_scores)
# active_count = calculate_active_patches_count_relative_threshold(current_patch_anomaly_scores, relative_threshold_percentage=RELATIVE_ACTIVE_PATCH_THRESHOLD_PERCENTAGE)
# top_1_avg = calculate_top_percent_average_anomaly(current_patch_anomaly_scores, top_percent=1)

# # Lógica de clasificación
# classification = 0
# if top_1_avg >= 0.30:
#     classification = 1
#     print(f"Clasificación: ANOMALÍA GRANDE (Top 1% Avg: {top_1_avg:.4f} >= 0.30)")
# elif 0.17 <= top_1_avg < 0.30:
#     print(f"Clasificación: Evaluación de anomalía leve/buena (Top 1% Avg: {top_1_avg:.4f})")
#     initial_classification_based_on_active_patches = 0
#     if active_count > 5:
#         initial_classification_based_on_active_patches = 1
#         print(f"-> ANOMALÍA LEVE (Parches Activos: {active_count} > 5)")
#         classification = 1
#     else:
#         print(f"-> Parches Activos ({active_count}) <= 5. Evaluando 'buena'.")
#         initial_classification_based_on_active_patches = 0
#         if dist_rms_median <= 0.055:
#             print(f"-> Condición Buena II (RMS - Mediana <= 0.055): True ({dist_rms_median:.4f})")
#             cond_I_met = spatial_var >= 5.5
#             print(f"-> Condición Buena I (Varianza Espacial >= 5.5): {'True' if cond_I_met else 'False'} ({spatial_var:.2f})")
#             cond_III_met = dist_rms_mad >= 0.21
#             print(f"-> Condición Buena III (RMS - MAD >= 0.21): {'True' if cond_III_met else 'False'} ({dist_rms_mad:.4f})")
#             if cond_I_met or cond_III_met:
#                 classification = 0
#                 print(f"-> IMAGEN BUENA")
#             else:
#                 classification = initial_classification_based_on_active_patches
#                 print(f"-> {'ANOMALÍA LEVE' if classification == 1 else 'IMAGEN BUENA'} (Revertiendo a Parches Activos)")
#         else:
#             classification = 1
#             print(f"-> ANOMALÍA LEVE")

# print(f"Clasificación Final para {base_image_name}: {'Anómala' if classification == 1 else 'Buena'}")

# # --- Detección y visualización de regiones de anomalía "fuertes" ---
# if classification == 1:
#     start_time_region_detection = time.time()
#     print("\n  ** Clasificada como ANÓMALA. Buscando regiones fuertes... **")
#     strong_anomaly_region_threshold = 0.75
#     binary_strong_anomaly_map = anomaly_map_final_for_regions > strong_anomaly_region_threshold
    
#     if not np.any(binary_strong_anomaly_map):
#         print(f"    No se encontraron píxeles por encima del umbral de {strong_anomaly_region_threshold}.")
#     else:
#         labeled_anomaly_regions = measure.label(binary_strong_anomaly_map)
#         region_properties = measure.regionprops(labeled_anomaly_regions)
#         detected_strong_anomaly_regions = []
#         min_region_pixel_area = 50
#         original_img_width, original_img_height = query_img_pil.size
#         scale_x = original_img_width / input_size
#         scale_y = original_img_height / input_size

#         for region in region_properties:
#             if region.area >= min_region_pixel_area:
#                 min_y, min_x, max_y, max_x = region.bbox
#                 scaled_min_x = int(np.clip(min_x * scale_x, 0, original_img_width))
#                 scaled_min_y = int(np.clip(min_y * scale_y, 0, original_img_height))
#                 scaled_max_x = int(np.clip(max_x * scale_x, 0, original_img_width))
#                 scaled_max_y = int(np.clip(max_y * scale_y, 0, original_img_height))
#                 region_width = scaled_max_x - scaled_min_x
#                 region_height = scaled_max_y - scaled_min_y
#                 if region_width > 0 and region_height > 0:
#                     detected_strong_anomaly_regions.append({
#                         'bbox': (scaled_min_x, scaled_min_y, region_width, region_height),
#                         'area_pixels': region.area
#                     })

#         if detected_strong_anomaly_regions:
#             plt.figure(figsize=(10, 8))
#             plt.imshow(query_img_pil)
#             plt.title(f'Imagen Anómala con Regiones Fuertes: {base_image_name.replace(".png", "")}')
#             plt.axis('off')
#             ax = plt.gca()
#             for region_info in detected_strong_anomaly_regions:
#                 bbox = region_info['bbox']
#                 if bbox[2] > 0 and bbox[3] > 0:
#                     rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3],
#                                              linewidth=3, edgecolor='lime', facecolor='none', linestyle='-', alpha=0.9)
#                     ax.add_patch(rect)
#             ax.add_patch(patches.Rectangle((0,0), 0.1, 0.1, linewidth=3, edgecolor='lime', facecolor='none', linestyle='-', alpha=0.9, label=f'Regiones Anómalas Fuertes'))
#             plt.legend()
#             strong_regions_overlay_output_filename = os.path.join(ANOMALY_REGIONS_SAVE_DIR, f'anomaly_regions_{base_image_name}')
#             plt.tight_layout()
#             plt.savefig(strong_regions_overlay_output_filename)
#             plt.close()
#             print(f"    Plot de regiones anómalas fuertes guardado en: {strong_regions_overlay_output_filename}")
#         else:
#             print("    No se detectaron regiones válidas para dibujar.")
#     end_time_region_detection = time.time()
#     print(f"  Tiempo para detección de regiones: {end_time_region_detection - start_time_region_detection:.4f} segundos.")
# else:
#     print(f"  Clasificada como BUENA. No se dibujarán regiones anómalas.")

########### FIn AnomalyDIno

directorio_imagenes = TRAIN_GOOD_DIR
plot_save_directory_on_server = PLOT_SAVE_ROOT_DIR

# --- Función para buscar imágenes similares usando KNN ---
def buscar_imagenes_similares_knn(query_feature_map, pre_flattened_features_bank, k=3, nombres_archivos=None):
    query_feat_flatten = query_feature_map.flatten().cpu().numpy()
    features_bank_for_knn = pre_flattened_features_bank.cpu().numpy() if isinstance(pre_flattened_features_bank, torch.Tensor) else pre_flattened_features_bank

    start_time_knn_dist = time.time()
    distances = euclidean_distances([query_feat_flatten], features_bank_for_knn)
    nearest_indices = np.argsort(distances[0])[:k]
    end_time_knn_dist = time.time()
    print(f"Tiempo para calcular distancias KNN: {end_time_knn_dist - start_time_knn_dist:.4f} segundos")

    imagenes_similares = []
    rutas_imagenes_similares = []
    if nombres_archivos:
        for idx in nearest_indices:
            imagenes_similares.append(nombres_archivos[idx])
            rutas_imagenes_similares.append(os.path.join(directorio_imagenes, nombres_archivos[idx]))
    else: # Fallback if no filenames provided (less common for this use case)
        for idx in nearest_indices:
            imagenes_similares.append(f"Imagen_Banco_{idx:03d}.png")
            rutas_imagenes_similares.append(os.path.join(directorio_imagenes, f"Imagen_Banco_{idx:03d}.png"))
    return imagenes_similares, rutas_imagenes_similares, end_time_knn_dist

# --- Búsqueda KNN ---
print("\nBuscando imágenes similares usando el banco pre-aplanado del Coreset...")
imagenes_similares, rutas_imagenes_similares, time_knn_dist = buscar_imagenes_similares_knn(
    query_lr_features, coreset_relevant_flat_features_bank, nombres_archivos=coreset_relevant_filenames
)

# print("Imágenes similares:", imagenes_similares)

# # --- Visualización de imágenes similares ---
# plt.figure(figsize=(15, 5))
# plt.subplot(1, len(rutas_imagenes_similares) + 1, 1)
# plt.imshow(query_img_pil)
# plt.title(f'Consulta:\n{base_image_name}')
# plt.axis('off')

# for j, ruta_imagen_similar in enumerate(rutas_imagenes_similares):
#     try:
#         img_similar = Image.open(ruta_imagen_similar).convert('RGB')
#         plt.subplot(1, len(rutas_imagenes_similares) + 1, j + 2)
#         plt.imshow(img_similar)
#         plt.title(f'Vecino {j + 1}\n({os.path.basename(ruta_imagen_similar)})')
#         plt.axis('off')
#     except Exception as e:
#         print(f"Error al cargar imagen similar {os.path.basename(ruta_imagen_similar)}: {e}")
#         plt.subplot(1, len(rutas_imagenes_similares) + 1, j + 2)
#         plt.text(0.5, 0.5, "Error de Carga", ha='center', va='center', transform=plt.gca().transAxes)
#         plt.title(f'Vecino {j + 1}\n(Error)')
#         plt.axis('off')

# output_similar_plot_filename = os.path.join(plot_save_directory_on_server, f'similar_images_plot_{base_image_name}')
# plt.tight_layout()
# plt.savefig(output_similar_plot_filename)
# plt.close()
# print(f"Plot de imágenes similares guardado en: {output_similar_plot_filename}")


# --- Aplicar FeatUp para obtener características de alta resolución ---
def apply_featup_hr(image_path, featup_upsampler, image_transform, device):
    image_pil = Image.open(image_path).convert("RGB")
    image_tensor = image_transform(image_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        lr_feats = featup_upsampler.model(image_tensor)
        hr_feats = featup_upsampler(image_tensor)
    return lr_feats.cpu(), hr_feats.cpu()

# Características de la imagen de consulta
input_query_tensor_original = transform(Image.open(query_image_path).convert("RGB")).unsqueeze(0).to(device)
query_lr_feats_featup, query_hr_feats = apply_featup_hr(query_image_path, upsampler, transform, device)

# plot_feats(unnorm(input_query_tensor_original)[0], query_lr_feats_featup[0], query_hr_feats[0])
# fig_query_feats = plt.gcf()
# fig_query_feats.suptitle(f'Características FeatUp: {base_image_name.replace(".png", "")}')
# output_query_feat_plot_filename = os.path.join(FEATUP_PLOTS_DIR, f'featup_query_image_features_plot_{base_image_name}')
# plt.tight_layout()
# fig_query_feats.savefig(output_query_feat_plot_filename)
# plt.close(fig_query_feats)
# print(f"Plot de características FeatUp (consulta) guardado en: {output_query_feat_plot_filename}")

# Características de las imágenes similares
similar_hr_feats_list = []
for j, similar_image_path in enumerate(rutas_imagenes_similares):
    input_similar_tensor_original = transform(Image.open(similar_image_path).convert("RGB")).unsqueeze(0).to(device)
    similar_lr_feats, similar_hr_feats = apply_featup_hr(similar_image_path, upsampler, transform, device)
    similar_hr_feats_list.append(similar_hr_feats)

    # plot_feats(unnorm(input_similar_tensor_original)[0], similar_lr_feats[0], similar_hr_feats[0])
    # fig_similar_feats = plt.gcf()
    # fig_similar_feats.suptitle(f'Características FeatUp Vecino {j + 1}: {os.path.basename(similar_image_path).replace(".png", "")}')
    # output_similar_feat_plot_filename = os.path.join(FEATUP_PLOTS_DIR, f'featup_similar_image_{j + 1}_features_plot_{base_image_name}')
    # plt.tight_layout()
    # fig_similar_feats.savefig(output_similar_feat_plot_filename)
    # plt.close(fig_similar_feats)
    # print(f"Plot de características FeatUp (vecino {j + 1}) guardado en: {output_similar_feat_plot_filename}")

################################
### Aplicando Máscaras SAM query y similares

#print(f"\nIniciando SAM para la imagen anómala: {base_image_name}")
start_time_sam = time.time()

# --- Funciones Auxiliares de Visualización ---
def show_mask(mask, ax, random_color=False, borders=True):
    color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0) if random_color else np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image_alpha = np.zeros((h, w, 4), dtype=np.float32)
    mask_image_alpha[mask > 0] = color
    if borders:
        mask_uint8 = mask.astype(np.uint8) * 255
        contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contour_image = np.zeros((h, w, 3), dtype=np.uint8)
        cv2.drawContours(contour_image, contours, -1, (255, 255, 255), thickness=2)
        contour_mask = (contour_image.astype(np.float32) / 255.0).sum(axis=-1) > 0
        mask_image_alpha[contour_mask > 0, :3] = 1.0
        mask_image_alpha[contour_mask > 0, 3] = 0.5
    ax.imshow(mask_image_alpha)

def show_points(coords, labels, ax, marker_size=375):
    ax.scatter(coords[labels==1][:, 0], coords[labels==1][:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(coords[labels==0][:, 0], coords[labels[0]==0][:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_masks_grid(image, masks, points=None, plot_title="Generated Masks", ax=None, num_masks=0):
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image)
    if points is not None:
        show_points(points, np.ones(points.shape[0], dtype=int), ax, marker_size=50)
    for mask_data in masks:
        show_mask(mask_data["segmentation"], ax, random_color=True)
    ax.set_title(f"{plot_title} (Masks: {num_masks})", fontsize=18)
    ax.axis('off')
# --- Fin Funciones Auxiliares ---

try:
    image_for_sam_np = np.array(Image.open(query_image_path).convert("RGB"))
    print(f"Dimensiones imagen SAM: {image_for_sam_np.shape}")
except Exception as e:
    print(f"Error procesando imagen para SAM: {e}. Saltando SAM.")
    sam2_model = None

if sam2_model is not None:
    points_grid_density = 16
    min_mask_area_pixels = 200.0
    max_mask_area_pixels = 450000.0

    mask_generator_query = SAM2AutomaticMaskGenerator(
        model=sam2_model,
        points_per_side=points_grid_density,
        points_per_batch=256,
        pred_iou_thresh=0.48,
        stability_score_thresh=0.7,
        crop_n_layers=0,
        min_mask_region_area=min_mask_area_pixels,
    )

    print(f"Generando máscaras para consulta con grid de {points_grid_density}x{points_grid_density} puntos...")
    masks_data_query_image = mask_generator_query.generate(image_for_sam_np)
    


    
    
    


    # --- Visualización de Máscaras Individuales ---
    # def visualize_individual_masks(image, masks, output_dir, filename_prefix):
    #     """
    #     Genera y guarda una gráfica que muestra las máscaras individuales superpuestas en la imagen original.
    #     """
    #     os.makedirs(output_dir, exist_ok=True)
    #     for i, mask_data in enumerate(masks):
    #         plt.figure(figsize=(8, 8))
    #         plt.imshow(image)
    #         show_mask(mask_data["segmentation"], plt.gca(), random_color=True)
    #         plt.axis('off')
    #         plt.title(f'Máscara Individual {i + 1}')
            
    #         # Guardar la gráfica
    #         output_path = os.path.join(output_dir, f"{filename_prefix}_mask_{i + 1}.png")
    #         plt.savefig(output_path, bbox_inches='tight')
    #         plt.close()
    #         print(f"Máscara individual {i + 1} guardada en: {output_path}")

    # # --- Generar y guardar las máscaras individuales ---
    # #PLOT_SAVE_ROOT_DIR = "/home/imercatoma/SAM_output/individual_masks"
    # INDIVIDUAL_MASKS_DIR = os.path.join(PLOT_SAVE_ROOT_DIR, "individual_masks")
    # visualize_individual_masks(image_for_sam_np, masks_data_query_image, INDIVIDUAL_MASKS_DIR, "query")
        

    print(f"Número de máscaras generadas para la imagen de consulta: {len(masks_data_query_image)}")
    #masks_data_query_image = [m for m in masks_data_query_image if m['area'] <= max_mask_area_pixels] filtrar fondo
    #print(f"Máscaras generadas para consulta: {len(masks_data_query_image)}.")

    mask_generator_similar = SAM2AutomaticMaskGenerator(
        model=sam2_model,
        points_per_side=points_grid_density,
        points_per_batch=256,
        pred_iou_thresh=0.48,
        stability_score_thresh=0.7,
        crop_n_layers=0,
        min_mask_region_area=min_mask_area_pixels,
    )

    print("\nGenerando máscaras SAM para imágenes similares...")
    similar_masks_raw_list = []
    for j, similar_image_path in enumerate(rutas_imagenes_similares):
        try:
            image_np_similar_for_sam = np.array(Image.open(similar_image_path).convert('RGB'))
            print(f"--- Procesando vecino {j+1}: {os.path.basename(similar_image_path)} ---")
            current_similar_masks_data = mask_generator_similar.generate(image_np_similar_for_sam)
            #current_similar_masks_data = [m for m in current_similar_masks_data if m['area'] <= max_mask_area_pixels] # filtrar area
            similar_masks_raw_list.append(current_similar_masks_data)
            print(f"Máscaras generadas para vecino {j+1}: {len(current_similar_masks_data)}.")
        except Exception as e:
            print(f"Error procesando imagen similar {os.path.basename(similar_image_path)} para SAM: {e}")

    end_time_sam = time.time()
    print(f"Tiempo total de ejecución de SAM: {end_time_sam - start_time_sam:.4f} segundos.")

    # print("\nGenerando visualización combinada de imágenes segmentadas...")
    # combined_plots_directory = os.path.join(PLOT_SAVE_ROOT_DIR, "combined_segmented_plots")
    # os.makedirs(combined_plots_directory, exist_ok=True)

    # def plot_combined_segmented(query_original_path, query_masks, similar_original_paths, similar_masks_list, output_dir, current_image_name):
    #     num_similar = len(similar_original_paths)
    #     if num_similar == 0: return

    #     total_subplots = 2 + num_similar
    #     fig, axes = plt.subplots(1, total_subplots, figsize=(5 * total_subplots, 6))

    #     try:
    #         query_img_orig = Image.open(query_original_path).convert('RGB')
    #         axes[0].imshow(query_img_orig)
    #         axes[0].set_title(f'Consulta Original:\n{current_image_name.replace(".png", "")}')
    #         axes[0].axis('off')
    #         show_masks_grid(np.array(query_img_orig), query_masks, plot_title=f'Consulta Segmentada', ax=axes[1], num_masks=len(query_masks))
    #     except Exception as e:
    #         print(f"Error al graficar imagen de consulta original/segmentada: {e}")
    #         axes[0].text(0.5, 0.5, "Error", ha='center', va='center', transform=axes[0].transAxes)
    #         axes[0].set_title('Consulta Original (Error)'); axes[0].axis('off')
    #         axes[1].text(0.5, 0.5, "Error", ha='center', va='center', transform=axes[1].transAxes)
    #         axes[1].set_title('Consulta Segmentada (Error)'); axes[1].axis('off')

    #     for j, similar_path in enumerate(similar_original_paths):
    #         if j + 2 >= total_subplots: break
    #         try:
    #             similar_img_orig = Image.open(similar_path).convert('RGB')
    #             current_similar_masks = similar_masks_list[j] if j < len(similar_masks_list) else []
    #             show_masks_grid(np.array(similar_img_orig), current_similar_masks,
    #                             plot_title=f'Vecino {j+1} Segmentado\n({os.path.basename(similar_path)})',
    #                             ax=axes[j + 2], num_masks=len(current_similar_masks))
    #         except Exception as e:
    #             print(f"Error al graficar imagen similar {os.path.basename(similar_path)}: {e}")
    #             axes[j + 2].text(0.5, 0.5, "Error", ha='center', va='center', transform=axes[j + 2].transAxes)
    #             axes[j + 2].set_title(f'Vecino {j+1} (Error)'); axes[j + 2].axis('off')

    #     plt.tight_layout()
    #     output_filename = os.path.join(output_dir, f'combined_query_and_similar_segmented_{current_image_name}')
    #     plt.savefig(output_filename)
    #     plt.close(fig)
    #     print(f"Plot combinado de imágenes segmentadas guardado en: {output_filename}")

    # plot_combined_segmented(
    #     query_image_path,
    #     masks_data_query_image,
    #     rutas_imagenes_similares,
    #     similar_masks_raw_list,
    #     combined_plots_directory,
    #     base_image_name
    # )
    
#print(f"La imagen {base_image_name} fue clasificada como BUENA o el modelo SAM no se pudo cargar. No se generarán máscaras SAM.")
print("\nAnálisis de detección de anomalías para una sola imagen completado.")



#######################

def process_masks_with_hierarchy(image, masks, output_dir, filename_prefix, overlap_threshold=0.8):
    os.makedirs(output_dir, exist_ok=True)
    
    processed_masks = []
    original_mask_segments = [mask_data["segmentation"] for mask_data in masks]

    for i, mask_data_a in enumerate(masks):
        mask_a_current_processing = np.copy(mask_data_a["segmentation"]) # Copia para evitar modificar el original
        is_completely_internal_to_another = False # Si la máscara A está completamente contenida en otra
        potential_holes_for_mask_a = [] # Máscaras que podrían crear un "hueco" en A

        for j, mask_data_b in enumerate(masks):
            if i == j:
                continue

            mask_b = original_mask_segments[j]

            # Si la máscara A está totalmente dentro de la máscara B
            if np.all(np.logical_and(mask_a_current_processing, mask_b) == mask_a_current_processing):
                is_completely_internal_to_another = True
                break # A es interna, no necesita ahuecarse

            # Si la máscara B está contenida o superpone significativamente a la máscara A
            intersection_ab = np.logical_and(mask_b, mask_a_current_processing)
            area_b = np.sum(mask_b)
            area_intersection_ab = np.sum(intersection_ab)

            # Considera B como un hueco si está contenida o si la superposición es significativa
            if area_b > 0 and (np.all(intersection_ab == mask_b) or (area_intersection_ab / area_b > overlap_threshold and area_intersection_ab > 0)):
                potential_holes_for_mask_a.append(mask_b)

        if is_completely_internal_to_another:
            # Si A es interna a otra, la añadimos sin cambios
            processed_masks.append(mask_data_a["segmentation"])
            display_title = f'Máscara {i + 1} (Interna - Sin cambios)'
        else:
            # Si A es externa, aplicamos los "huecos"
            hollowed = False
            for hole_mask in potential_holes_for_mask_a:
                mask_a_current_processing = np.logical_and(mask_a_current_processing, np.logical_not(hole_mask))
                hollowed = True
            
            processed_masks.append(mask_a_current_processing)
            if hollowed:
                display_title = f'Máscara {i + 1} (Externa - Hueca)'
            else:
                display_title = f'Máscara {i + 1} (Externa - Sin huecos significativos)'

        plt.figure(figsize=(8, 8))
        plt.imshow(image)
        show_mask(processed_masks[-1], plt.gca(), random_color=True)
        plt.axis('off')
        plt.title(display_title)
            
        output_path = os.path.join(output_dir, f"{filename_prefix}_processed_mask_{i + 1}.png")
        plt.savefig(output_path, bbox_inches='tight')
        plt.close()
        print(f"Máscara procesada {i + 1} guardada en: {output_path}")

    return [{"segmentation": mask.astype(np.float32)} for mask in processed_masks]

# Directorio para guardar las máscaras procesadas
PROCESSED_MASKS_DIR = os.path.join(PLOT_SAVE_ROOT_DIR, "processed_masks")

# Llamar a la función para procesar las máscaras
processed_masks = process_masks_with_hierarchy(image_for_sam_np, masks_data_query_image, PROCESSED_MASKS_DIR, "query")

masks_data_query_image = processed_masks
print("Shape de masks_data_query_image:", len(masks_data_query_image))
###############





# --- Implementación del punto 3.4.3. Object Feature Map ---
import torch.nn.functional as F # Importa F para F.interpolate
from sklearn.decomposition import PCA
import os

def process_masks_to_object_feature_maps(raw_masks, hr_feature_map, target_h, target_w, sam_processed_image_shape):
    """
    Procesa una lista de máscaras de SAM para obtener mapas de características de objeto.
    Args:
        raw_masks (list): Lista de diccionarios de máscaras crudas de SAM.
                          Cada dict tiene una clave 'segmentation' (np.ndarray booleana).
        hr_feature_map (torch.Tensor): Mapa de características de alta resolución (C, 8H', 8W').
                                        Debe ser de la imagen correspondiente (query o reference).
                                        Asegúrate de que ya esté en el dispositivo correcto.
        target_h (int): Altura objetivo para la máscara escalada (8H').
        target_w (int): Ancho objetivo para la máscara escalada (8W').
        sam_processed_image_shape (tuple): La forma (H, W, C) de la imagen a la que SAM se aplicó
                                            para generar las máscaras (ej. (1024, 1024, 3)).
                                            Esto es crucial para escalar correctamente la máscara.
    Returns:
        torch.Tensor: Tensor de mapas de características de objeto (M, C, 8H', 8W').
                      Si no hay máscaras, devuelve un tensor vacío (0, C, 8H', 8W').
    """
    if not raw_masks:
        print("Advertencia: No se encontraron máscaras para procesar. Devolviendo tensor vacío.")
        C_dim = hr_feature_map.shape[0] if hr_feature_map.ndim >=3 else 0
        return torch.empty(0, C_dim, target_h, target_w, device=hr_feature_map.device)

    object_feature_maps_list = []
    scaled_mask_append = []
    C_dim = hr_feature_map.shape[0] # Número de canales de las características HR

    for mask_info in raw_masks:
        # Convertir la máscara booleana de numpy a tensor float y añadir dimensiones de lote y canal
        mask_np = mask_info['segmentation'].astype(np.float32)
        mask_tensor_original_res = torch.from_numpy(mask_np).unsqueeze(0).unsqueeze(0) # (1, 1, H_orig, W_orig)
        # Mover la máscara al mismo dispositivo que el mapa de características HR
        mask_tensor_original_res = mask_tensor_original_res.to(hr_feature_map.device)

        # 1. Escalar la máscara a (8H', 8W') usando interpolación bilineal
        scaled_mask = F.interpolate(mask_tensor_original_res,
                                     size=(target_h, target_w),
                                     mode='bilinear',
                                     align_corners=False)
        # Opcional: Binarizar la máscara después del escalado si se requiere una máscara estricta (0 o 1)
        scaled_mask = (scaled_mask > 0.5).float()
        # Append scaled mask to the list
        scaled_mask_append.append(scaled_mask)
        
        # 2. Multiplicación elemento a elemento con el mapa de características HR
        if hr_feature_map.ndim == 3:
            hr_feature_map_with_batch = hr_feature_map.unsqueeze(0) # -> (1, C, H, W)
        else: # Si ya es (1, C, H, W)
            hr_feature_map_with_batch = hr_feature_map

        object_feature_map_i = scaled_mask * hr_feature_map_with_batch
        object_feature_maps_list.append(object_feature_map_i)

    # Concatenar todos los mapas de características de objeto
    final_object_feature_maps = torch.cat(object_feature_maps_list, dim=0) # (M, C, 8H', 8W')
    final_scaled_masks = torch.cat(scaled_mask_append, dim=0)
    # Save scaled masks outside the function
    global saved_scaled_masks
    saved_scaled_masks = final_scaled_masks
    return final_object_feature_maps, final_scaled_masks






# --- Visualización de Mapas de Características de Objeto ---
def visualize_object_feature_map(original_image_path, sam_mask_info, hr_feature_map_tensor,
                                   object_feature_map_tensor, target_h, target_w,
                                   plot_save_dir, plot_filename_prefix, mask_idx,
                                   sam_processed_image_shape):
    """
    Genera y guarda una visualización de un mapa de características de objeto.
    Muestra la imagen original, la máscara de SAM y el mapa de características de objeto.
    """
    try:
        original_img = Image.open(original_image_path).convert("RGB")
        sam_mask_np = sam_mask_info['segmentation']

        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # Plot 1: Imagen Original
        axes[0].imshow(original_img)
        axes[0].set_title(f'Imagen Original\n{os.path.basename(original_image_path)}')
        axes[0].axis('off')

        # Plot 2: Máscara SAM (escalada para visualización si es necesario, pero manteniendo la forma original)
        # We need to scale the SAM mask to the input_size for direct overlay if original_img is resized,
        # but the mask itself comes from the SAM processed image which might be 1024x1024.
        # For display simplicity, we'll just show the original mask over the original image,
        # ensuring the aspect ratio aligns.
        mask_display = sam_mask_np # Boolean mask
        axes[1].imshow(original_img) # Overlay on original
        # For plotting mask, we scale it to match the original image's aspect ratio/size if necessary for correct overlay
        # Since SAM masks are usually for specific input sizes (e.g., 1024x1024), we should ensure it fits.
        # However, for simplicity here, we assume the mask is compatible or will be interpolated by imshow.
        show_mask(mask_display, axes[1], random_color=False, borders=True) # Use the show_mask helper
        axes[1].set_title(f'Máscara SAM {mask_idx}')
        axes[1].axis('off')

        # Plot 3: Object Feature Map (visualización de PCA)
        # Reshape C, H, W to (H*W, C) for PCA
        if object_feature_map_tensor.numel() == 0:
            axes[2].text(0.5, 0.5, "No hay características de objeto", ha='center', va='center', transform=axes[2].transAxes)
            axes[2].set_title('Mapa de Características de Objeto (Vacío)')
            axes[2].axis('off')
        else:
            ofm_cpu = object_feature_map_tensor.squeeze().cpu().numpy() # Remove batch dim if present
            if ofm_cpu.ndim == 3: # C, H, W
                C, H, W = ofm_cpu.shape
                ofm_reshaped = ofm_cpu.transpose(1, 2, 0).reshape(-1, C) # H*W, C

                if C > 3: # Apply PCA if more than 3 channels
                    pca = PCA(n_components=3)
                    ofm_pca = pca.fit_transform(ofm_reshaped)
                    # Normalize PCA results to [0, 1] for image display
                    ofm_pca_normalized = (ofm_pca - ofm_pca.min()) / (ofm_pca.max() - ofm_pca.min() + 1e-8)
                    ofm_display = ofm_pca_normalized.reshape(H, W, 3)
                    axes[2].imshow(ofm_display)
                    axes[2].set_title(f'Mapa de Características de Objeto (PCA)\nMáscara {mask_idx}')
                else: # If 1, 2, or 3 channels, display directly (grayscale or RGB)
                    if C == 1:
                        ofm_display = ofm_cpu.squeeze()
                        axes[2].imshow(ofm_display, cmap='viridis')
                    elif C == 3:
                        ofm_display = ofm_cpu.transpose(1, 2, 0) # H, W, C
                        ofm_display_norm = (ofm_display - ofm_display.min()) / (ofm_display.max() - ofm_display.min() + 1e-8)
                        axes[2].imshow(ofm_display_norm)
                    else: # 2 channels, or other, might not display well as RGB. Use grayscale of first channel.
                        ofm_display = ofm_cpu[0]
                        axes[2].imshow(ofm_display, cmap='viridis')
                    axes[2].set_title(f'Mapa de Características de Objeto\nMáscara {mask_idx}')
            else: # If the object_feature_map_tensor somehow resulted in a non-3D tensor for a single mask
                axes[2].text(0.5, 0.5, "Formato de características de objeto inesperado", ha='center', va='center', transform=axes[2].transAxes)
                axes[2].set_title('Mapa de Características de Objeto (Error)')

            axes[2].axis('off')

        plt.tight_layout()
        save_path = os.path.join(plot_save_dir, f"{plot_filename_prefix}_mask_{mask_idx}.png")
        plt.savefig(save_path)
        plt.close(fig)
        # print(f"Visualización del mapa de características de objeto guardada en: {save_path}")

    except Exception as e:
        print(f"Error al visualizar el mapa de características de objeto para máscara {mask_idx} de {os.path.basename(original_image_path)}: {e}")

# --- Aplicar el proceso a la imagen de consulta y a las imágenes de referencia ---

print("\n--- Generando Mapas de Características de Objeto ---")

# Dimensiones objetivo para las máscaras después de escalar (8H', 8W')
TARGET_MASK_H = 8 * H_prime # 8 * 16 = 128
TARGET_MASK_W = 8 * W_prime # 8 * 16 = 128
print(f"TARGET_MASK_H: {TARGET_MASK_H}")
print(f"TARGET_MASK_W: {TARGET_MASK_W}")

# # Para la imagen de consulta (Iq)
# fobj_q = process_masks_to_object_feature_maps(
#     masks_data_query_image,
#     query_hr_feats.squeeze(0), # Pasamos (C, 8H', 8W') para que la función maneje el batch
#     TARGET_MASK_H,
#     TARGET_MASK_W,
#     image_for_sam_np.shape # Pasamos la forma real de la imagen que SAM procesó
# ).to(device) # Mover a la GPU si no está ya

fobj_q, scaled_masks = process_masks_to_object_feature_maps(
    masks_data_query_image,
    query_hr_feats.squeeze(0), # Pasamos (C, 8H', 8W') para que la función maneje el batch
    TARGET_MASK_H,
    TARGET_MASK_W,
    image_for_sam_np.shape # Pasamos la forma real de la imagen que SAM procesó
)

# Mover el tensor `fobj_q` a la GPU si no está ya
fobj_q = fobj_q.to(device)


print(f"Dimensiones de fobj_q (Mapas de Características de Objeto de Iq): {fobj_q.shape}") # Esperado (M, 384, 128, 128)

# Para las imágenes de referencia (Ir)
all_fobj_r_list = [] # Para almacenar fobj_r para cada imagen similar
for i, similar_hr_feats in enumerate(similar_hr_feats_list):
    current_similar_masks_raw = similar_masks_raw_list[i]
    # Necesitamos obtener la forma original de la imagen similar para SAM
    img_similar_pil = Image.open(rutas_imagenes_similares[i]).convert('RGB') # Cargar de nuevo para obtener su forma
    image_np_similar_for_sam_shape = np.array(img_similar_pil).shape

    fobj_r_current, scaled_masks = process_masks_to_object_feature_maps(
        current_similar_masks_raw,
        similar_hr_feats.squeeze(0), # Pasamos (C, 8H', 8W')
        TARGET_MASK_H,
        TARGET_MASK_W,
        image_np_similar_for_sam_shape # Pasamos la forma real de la imagen que SAM procesó
    )
    fobj_r_current = fobj_r_current.to(device)
    
    all_fobj_r_list.append(fobj_r_current)
    print(f"Dimensiones de fobj_r para vecino {i+1}: {fobj_r_current.shape}") # Esperado (N, 384, 128, 128)
    # Imprimir el tipo de cada elemento en all_fobj_r_list
    print("\nTipos de los elementos en all_fobj_r_list:")
    for idx, fobj_r in enumerate(all_fobj_r_list):
        print(f"Vecino {idx + 1}: Tipo de fobj_r:", type(fobj_r))
print("\nProceso de 'Object Feature Map' completado. ¡Ahora tienes los fobj_q y fobj_r listos!")



# --- Directorio para guardar plots de Mapas de Características de Objeto ---
OFM_PLOTS_DIR = os.path.join(PLOT_SAVE_ROOT_DIR, "object_feature_map_plots")
os.makedirs(OFM_PLOTS_DIR, exist_ok=True)
print(f"\nLos plots de Mapas de Características de Objeto se guardarán en: {OFM_PLOTS_DIR}")

# Visualización para la imagen de consulta (Iq)
print("\nGenerando visualizaciones de Mapas de Características de Objeto para la consulta...")
for i, mask_info in enumerate(masks_data_query_image):
    # fobj_q es (M, C, H, W). Necesitamos una máscara a la vez.
    if i < fobj_q.shape[0]: # Asegurarse de que tenemos un OFM para esta máscara
        visualize_object_feature_map(
            query_image_path,
            mask_info,
            query_hr_feats, # Pasamos el HR feature map completo
            fobj_q[i].unsqueeze(0), # Pasamos solo el OFM de la máscara actual, con batch dim para la función
            TARGET_MASK_H,
            TARGET_MASK_W,
            OFM_PLOTS_DIR,
            f"query_{base_image_name.replace('.png', '')}",
            i,
            image_for_sam_np.shape
        )
    else:
        print(f"Advertencia: No se encontró OFM para la máscara de consulta {i}.")

# Visualización para las imágenes de referencia (Ir)
print("\nGenerando visualizaciones de Mapas de Características de Objeto para los vecinos...")
for i, similar_image_path in enumerate(rutas_imagenes_similares):
    current_similar_masks_raw = similar_masks_raw_list[i]
    current_similar_hr_feats = similar_hr_feats_list[i]
    current_fobj_r = all_fobj_r_list[i]
    img_similar_pil_for_shape = Image.open(similar_image_path).convert('RGB')
    image_np_similar_for_sam_shape = np.array(img_similar_pil_for_shape).shape

    if not current_fobj_r.numel() == 0: # Solo procesar si hay OFMs generados para este vecino
        for j, mask_info in enumerate(current_similar_masks_raw):
            if j < current_fobj_r.shape[0]: # Asegurarse de que tenemos un OFM para esta máscara
                visualize_object_feature_map(
                    similar_image_path,
                    mask_info,
                    current_similar_hr_feats,
                    current_fobj_r[j].unsqueeze(0), # OFM de la máscara actual
                    TARGET_MASK_H,
                    TARGET_MASK_W,
                    OFM_PLOTS_DIR,
                    f"neighbor_{i+1}_{os.path.basename(similar_image_path).replace('.png', '')}",
                    j,
                    image_np_similar_for_sam_shape
                )
            else:
                print(f"Advertencia: No se encontró OFM para la máscara {j} del vecino {i+1}.")
    else:
        print(f"No se generaron OFMs para el vecino {i+1} ({os.path.basename(similar_image_path)}), saltando visualización.")

print("\nVisualización de Mapas de Características de Objeto completada.")



Cargando datos del coreset...
Coreset cargado. Dimensión: torch.Size([10009, 384])
NearestNeighbors finder inicializado.
Cargando modelo DINOv2...


Using cache found in /home/imercatoma/.cache/torch/hub/facebookresearch_dinov2_main


Modelo DINOv2 cargado.
Cargando modelo SAM2 desde /home/imercatoma/sam2_repo_independent/checkpoints/sam2.1_hiera_small.pt...
Modelo SAM2 cargado.

--- Procesando imagen: 001.png ---

Buscando imágenes similares usando el banco pre-aplanado del Coreset...
Tiempo para calcular distancias KNN: 1.0846 segundos
Dimensiones imagen SAM: (1024, 1024, 3)
Generando máscaras para consulta con grid de 16x16 puntos...
Número de máscaras generadas para la imagen de consulta: 7
Máscaras generadas para consulta: 6.

Generando máscaras SAM para imágenes similares...
--- Procesando vecino 1: 036.png ---
Máscaras generadas para vecino 1: 2.
--- Procesando vecino 2: 064.png ---
Máscaras generadas para vecino 2: 2.
--- Procesando vecino 3: 092.png ---
Máscaras generadas para vecino 3: 2.
Tiempo total de ejecución de SAM: 23.4892 segundos.

Análisis de detección de anomalías para una sola imagen completado.
Máscara procesada 1 guardada en: /home/imercatoma/FeatUp/plots_single_3/cut_001/processed_masks/quer

In [10]:
# -----------3.5.2 Object matching module-----------------
## Matching
# --- Definición de la función show_anomalies_on_image ---
def show_anomalies_on_image(image_np, masks, anomalous_info, alpha=0.5, save_path=None):

    plt.figure(figsize=(8, 8))
    plt.imshow(image_np)

    for obj_id, similarity in anomalous_info: # Iterate through (id, similarity) tuples
        # Extraer la máscara binaria real
        mask = masks[obj_id]['segmentation']
        if isinstance(mask, torch.Tensor):
            mask = mask.cpu().numpy()

        # Crear máscara en rojo
        colored_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
        colored_mask[mask > 0] = [255, 0, 0]
        plt.imshow(colored_mask, alpha=alpha)

        # Calcular centroide para colocar el texto
        ys, xs = np.where(mask > 0)
        if len(xs) > 0 and len(ys) > 0:
            cx = int(xs.mean())
            cy = int(ys.mean())
            
            # Create text with index and percentage
            text_label = f"{obj_id} ({similarity*100:.2f}%)"
            plt.text(cx, cy, text_label, color='white', fontsize=10, fontweight='bold', ha='center', va='center',
                     bbox=dict(facecolor='red', alpha=0.6, edgecolor='none', boxstyle='round,pad=0.2'))

    plt.title("Objetos Anómalos en Rojo con Índice y Similitud") # Updated title for clarity
    plt.axis("off")

    if save_path:
        plt.tight_layout()
        plt.savefig(save_path)
        print(f"✅ Plot de anomalías guardado en: {save_path}")

    plt.show()
    plt.close()
# --- Fin de la definición de la función show_anomalies_on_image ---
# --- Nuevas funciones de ploteo para la matriz P y P_augmented_full ---
def plot_assignment_matrix(P_matrix, query_labels, reference_labels, save_path=None, title="Matriz de Asignación P"):
    """
    Visualiza la matriz de asignación P como un mapa de calor.

    Args:
        P_matrix (torch.Tensor or np.array): La matriz de asignación (M x N).
        query_labels (list): Etiquetas para los objetos de consulta (eje Y).
        reference_labels (list): Etiquetas para los objetos de referencia (eje X).
        save_path (str, optional): Ruta para guardar la imagen del plot.
        title (str): Título del plot.
    """
    if isinstance(P_matrix, torch.Tensor):
        #P_matrix = P_matrix.cpu().numpy()
        P_matrix = P_matrix.detach().cpu().numpy()

    plt.figure(figsize=(P_matrix.shape[1] * 0.8 + 2, P_matrix.shape[0] * 0.8 + 2))
    plt.imshow(P_matrix, cmap='viridis', origin='upper', aspect='auto')
    plt.colorbar(label='Probabilidad de Asignación')
    plt.xticks(np.arange(len(reference_labels)), reference_labels, rotation=45, ha="right")
    plt.yticks(np.arange(len(query_labels)), query_labels)
    plt.xlabel('Objetos de Referencia')
    plt.ylabel('Objetos de Consulta')
    plt.title(title)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
        print(f"✅ Plot de la matriz de asignación guardado en: {save_path}")
    plt.show()
    plt.close()

def plot_augmented_assignment_matrix(P_augmented_full, query_labels, reference_labels, save_path=None, title="Matriz de Asignación Aumentada (con Trash Bin)"):
    """
    Visualiza la matriz de asignación aumentada (incluyendo los trash bins) como un mapa de calor.

    Args:
        P_augmented_full (torch.Tensor or np.array): La matriz de asignación aumentada ((M+1) x (N+1)).
        query_labels (list): Etiquetas para los objetos de consulta.
        reference_labels (list): Etiquetas para los objetos de referencia.
        save_path (str, optional): Ruta para guardar la imagen del plot.
        title (str): Título del plot.
    """
    if isinstance(P_augmented_full, torch.Tensor):
        #P_augmented_full = P_augmented_full.cpu().numpy()
        P_augmented_full = P_augmented_full.detach().cpu().numpy()

    # Añadir etiquetas para los trash bins
    full_query_labels = [f"Q_{i}" for i in query_labels] + ["Trash Bin (Q)"]
    full_reference_labels = [f"R_{i}" for i in reference_labels] + ["Trash Bin (R)"]

    plt.figure(figsize=(P_augmented_full.shape[1] * 0.8 + 2, P_augmented_full.shape[0] * 0.8 + 2))
    plt.imshow(P_augmented_full, cmap='viridis', origin='upper', aspect='auto')
    plt.colorbar(label='Probabilidad de Asignación')
    plt.xticks(np.arange(len(full_reference_labels)), full_reference_labels, rotation=45, ha="right")
    plt.yticks(np.arange(len(full_query_labels)), full_query_labels)
    plt.xlabel('Objetos de Referencia y Trash Bin')
    plt.ylabel('Objetos de Consulta y Trash Bin')
    plt.title(title)
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
        print(f"✅ Plot de la matriz de asignación aumentada guardado en: {save_path}")
    plt.show()
    plt.close()


# --- Fin de las nuevas funciones de ploteo ---

## Matching-continue---
## Matching
start_time_sam_matching = time.time()



import torch
import torch.nn as nn
import torch.nn.functional as F

def apply_global_max_pool(feat_map):
    return F.adaptive_max_pool2d(feat_map, output_size=1).squeeze(-1).squeeze(-1)

class SimpleObjectMatchingModule(nn.Module):
    def __init__(self, sinkhorn_iterations=100, sinkhorn_epsilon=0.1, bin_score_value=0.5):
        super(SimpleObjectMatchingModule, self).__init__()
        self.sinkhorn_iterations = sinkhorn_iterations
        self.sinkhorn_epsilon = sinkhorn_epsilon
        self.z = nn.Parameter(torch.tensor(bin_score_value, dtype=torch.float32))

    def forward(self, d_M_q, d_N_r):
        M = d_M_q.shape[0]
        N = d_N_r.shape[0]

        if M == 0 or N == 0:
            return torch.empty(M, N, device=d_M_q.device), \
                   torch.empty(M+1, N+1, device=d_M_q.device)

        score_matrix = torch.mm(d_M_q, d_N_r.T)
        #print("score_matrix (antes de Sinkhorn):\n", score_matrix)

        S_augmented = torch.zeros((M + 1, N + 1), device=d_M_q.device, dtype=d_M_q.dtype)
        S_augmented[:M, :N] = score_matrix
        S_augmented[:M, N] = self.z
        S_augmented[M, :N] = self.z
        S_augmented[M, N] = self.z
        print("S_augmented antes de Sinkhorn:\n", S_augmented)

        K = torch.exp(S_augmented / self.sinkhorn_epsilon)
        print("K (antes de Sinkhorn):\n", K)
        

        for i in range(self.sinkhorn_iterations):
            K = K / K.sum(dim=1, keepdim=True)
            K = K / K.sum(dim=0, keepdim=True)
            #print(f"Iteración {i+1}: K.shape = {K}")

        P_augmented_full = K
        P = P_augmented_full[:M, :N]

        return P, P_augmented_full

fobj_q_pooled = apply_global_max_pool(fobj_q)
print("Shape de fobj_q_pooled:", fobj_q_pooled.shape)
print("Máximo de fobj_q_pooled:", torch.max(fobj_q_pooled).item())
print("Mínimo de fobj_q_pooled:", torch.min(fobj_q_pooled).item())

all_fobj_r_pooled_list = []
for fobj_r_current in all_fobj_r_list:
    pooled_r = apply_global_max_pool(fobj_r_current)
    all_fobj_r_pooled_list.append(pooled_r)
    
d_M_q = F.normalize(fobj_q_pooled, p=2, dim=1) #shape (M, C)
d_N_r_list = [F.normalize(fobj_r_pooled, p=2, dim=1) 
                              for fobj_r_pooled in all_fobj_r_pooled_list]
print("Máximo de d_M_q:", torch.max(d_M_q).item())
print("Mínimo de d_M_q:", torch.min(d_M_q).item())

object_matching_module = SimpleObjectMatchingModule(
    sinkhorn_iterations=50,
    sinkhorn_epsilon=0.1,
    bin_score_value=2.36
).to(device)

P_matrices = []
P_augmented_full_matrices = []

for i, d_N_r_current_image in enumerate(d_N_r_list):
    d_M_q_cuda = d_M_q.to(device)
    d_N_r_current_image_cuda = d_N_r_current_image.to(device)

    P_current, P_augmented_current = object_matching_module(d_M_q_cuda, d_N_r_current_image_cuda)
    P_matrices.append(P_current)
    P_augmented_full_matrices.append(P_augmented_current)


print("\n--- Matrices P y P_augmented_full generadas ---")
# --- NUEVOS DICCIONARIOS CONSOLIDADOS ---
# Almacenarán para cada query_idx, las referencias que le corresponden de TODOS los vecinos.
M = d_M_q.shape[0]
all_matched_ref_indices_by_query_obj = {q_idx: [] for q_idx in range(M)} # M es el número de objetos de consulta (Iq)
all_closest_unmatched_ref_indices_by_query_obj = {q_idx: [] for q_idx in range(M)}
# Imprimir shapes de los diccionarios consolidados
#//////
print("\n--- Resultados Consolidados ---")
print("all_matched_ref_indices_by_query_obj:")
for q_idx, matches in all_matched_ref_indices_by_query_obj.items():
    print(f"  Objeto de Consulta {q_idx}: {matches}")

print("\nall_closest_unmatched_ref_indices_by_query_obj:")
for q_idx, closest_unmatches in all_closest_unmatched_ref_indices_by_query_obj.items():
    print(f"  Objeto de Consulta {q_idx}: {closest_unmatches}")
#/////////////////
# Procesar matrices P y P_augmented_full para obtener índices
for i, (P, P_augmented_full) in enumerate(zip(P_matrices, P_augmented_full_matrices)):
    current_neighbor_key = f"Vecino_{i+1}"
    N_current = P.shape[1] 

    print(f"\n--- Vecino {current_neighbor_key} ---")
    print(f"Matriz P (MxN) para el vecino {current_neighbor_key}:")
    print(P)
    print(f"Matriz P_augmented_full (M+1 x N+1) para el vecino {current_neighbor_key}:")
    print(P_augmented_full)

    # Imprimir sumas de filas y columnas de P_augmented_full
    augmented_with_totals = torch.cat([
        torch.cat([P_augmented_full, P_augmented_full.sum(dim=0, keepdim=True)], dim=0),
        torch.cat([P_augmented_full.sum(dim=1, keepdim=True), P_augmented_full.sum().unsqueeze(0).unsqueeze(0)], dim=0)
    ], dim=1)
    print(f"Matriz P_augmented_full con totales (M+2 x N+2):\n{augmented_with_totals}")

    print(f"\n--- Decisiones de Emparejamiento para el Vecino {current_neighbor_key} ---")
    for obj_idx in range(P.shape[0]):
        
        # Obtener la probabilidad más alta dentro de P y su índice
        if N_current > 0:
            max_prob_P = P[obj_idx].max().item()
            max_idx_P = P[obj_idx].argmax().item()
        else:
            max_prob_P = -float('inf')
            max_idx_P = -1


        trash_bin_prob = P_augmented_full[obj_idx, -1].item() 

        print(f"   Objeto de Consulta {obj_idx}:")
        print(f"     Probabilidad máxima en P: {max_prob_P:.4f} en el índice {max_idx_P}")
        print(f"     Probabilidad en el 'Trash Bin': {trash_bin_prob:.4f}")


       # Decisión y almacenamiento en los diccionarios consolidados
        if trash_bin_prob > max_prob_P:
            # Desemparejado: ahora añadimos el 'primer máximo' a la lista de ese objeto de consulta
            if max_idx_P != -1: # Solo añadir si hay un 'primer máximo' válido
                all_closest_unmatched_ref_indices_by_query_obj[obj_idx].append((i, max_idx_P)) # (índice_vecino, índice_referencia)
            print(f"     Decisión: DESEMPAREJADO. 'Casi-par' (PRIMER más similar en P): objeto {max_idx_P}")
        else:
            # Emparejado: añadir el emparejamiento real a la lista de ese objeto de consulta
            all_matched_ref_indices_by_query_obj[obj_idx].append((i, max_idx_P)) # (índice_vecino, índice_referencia)
            print(f"     Decisión: EMPAREJADO con objeto de imagen {max_idx_P}")


# --- Resultados Finales Consolidados ---
print("\n--- Resultados Finales Consolidados (Índices) ---")
print("all_matched_ref_indices_by_query_obj (query_idx: [(vecino_idx, ref_idx), ...]):")
for q_idx, matches in all_matched_ref_indices_by_query_obj.items():
    print(f"  Query {q_idx}: {matches}")

print("\nall_closest_unmatched_ref_indices_by_query_obj (query_idx: [(vecino_idx, second_ref_idx), ...]):")
for q_idx, closest_unmatches in all_closest_unmatched_ref_indices_by_query_obj.items():
    print(f"  Query {q_idx}: {closest_unmatches}")


Shape de fobj_q_pooled: torch.Size([6, 384])
Máximo de fobj_q_pooled: 5.209421157836914
Mínimo de fobj_q_pooled: 0.0
Máximo de d_M_q: 0.23235014081001282
Mínimo de d_M_q: 0.0
S_augmented antes de Sinkhorn:
 tensor([[0.8560, 0.9546, 2.3600],
        [0.9638, 0.8476, 2.3600],
        [0.7423, 0.8525, 2.3600],
        [0.7580, 0.8661, 2.3600],
        [0.8452, 0.9654, 2.3600],
        [0.7624, 0.8751, 2.3600],
        [2.3600, 2.3600, 2.3600]], device='cuda:0', grad_fn=<CopySlices>)
K (antes de Sinkhorn):
 tensor([[5.2205e+03, 1.3982e+04, 1.7756e+10],
        [1.5342e+04, 4.7985e+03, 1.7756e+10],
        [1.6743e+03, 5.0400e+03, 1.7756e+10],
        [1.9594e+03, 5.7723e+03, 1.7756e+10],
        [4.6826e+03, 1.5591e+04, 1.7756e+10],
        [2.0477e+03, 6.3175e+03, 1.7756e+10],
        [1.7756e+10, 1.7756e+10, 1.7756e+10]], device='cuda:0',
       grad_fn=<ExpBackward0>)
S_augmented antes de Sinkhorn:
 tensor([[0.8748, 0.9343, 2.3600],
        [0.9416, 0.8438, 2.3600],
        [0.7668, 0.8

In [11]:

# --- Función para graficar objetos de consulta y sus coincidencias ---
import matplotlib.pyplot as plt

def plot_matched_and_unmatched_objects(query_image_path, neighbor_image_paths, query_masks, neighbor_masks_list, matched_indices, unmatched_indices, output_dir):
    """
    Grafica los objetos de consulta y sus coincidencias (matched/unmatched) con los vecinos.

    Args:
        query_image_path (str): Ruta de la imagen de consulta.
        neighbor_image_paths (list): Lista de rutas de imágenes de vecinos.
        query_masks (list): Máscaras de objetos de consulta.
        neighbor_masks_list (list): Lista de listas de máscaras de objetos de vecinos.
        matched_indices (dict): Diccionario con índices de objetos emparejados.
        unmatched_indices (dict): Diccionario con índices de objetos no emparejados.
        output_dir (str): Directorio para guardar los gráficos.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Cargar imagen de consulta
    query_image = np.array(Image.open(query_image_path).convert("RGB"))

    # Cargar imágenes de vecinos
    neighbor_images = [np.array(Image.open(path).convert("RGB")) for path in neighbor_image_paths]

    # Iterar sobre cada objeto de consulta
    for obj_idx, query_mask in enumerate(query_masks):
        fig, axes = plt.subplots(1, len(neighbor_images) + 1, figsize=(15, 5))

        # Color único para el objeto actual
        color = np.random.random(3)

        # Graficar la imagen de consulta con el objeto actual
        axes[0].imshow(query_image)
        axes[0].set_title(f"Consulta - Objeto Q{obj_idx}")
        axes[0].axis("off")
        show_mask(query_mask["segmentation"], axes[0], random_color=False)
        axes[0].text(query_mask["segmentation"].shape[1] // 2, query_mask["segmentation"].shape[0] // 2, f"Q{obj_idx}", color=color, fontsize=12, fontweight="bold")

        # Graficar las imágenes de vecinos con los objetos emparejados o no emparejados
        for neighbor_idx, (neighbor_image, neighbor_masks) in enumerate(zip(neighbor_images, neighbor_masks_list)):
            axes[neighbor_idx + 1].imshow(neighbor_image)
            axes[neighbor_idx + 1].axis("off")

            # Título dependiendo de si es matched o unmatched
            if any(match[0] == neighbor_idx for match in matched_indices[obj_idx]):
                axes[neighbor_idx + 1].set_title(f"Vecino {neighbor_idx + 1} - Matched")
                for match in matched_indices[obj_idx]:
                    if match[0] == neighbor_idx:
                        neighbor_mask = neighbor_masks[match[1]]
                        show_mask(neighbor_mask["segmentation"], axes[neighbor_idx + 1], random_color=False)
                        axes[neighbor_idx + 1].text(neighbor_mask["segmentation"].shape[1] // 2, neighbor_mask["segmentation"].shape[0] // 2, f"N{match[1]}", color=color, fontsize=12, fontweight="bold")
            else:
                axes[neighbor_idx + 1].set_title(f"Vecino {neighbor_idx + 1} - Unmatched")
                for unmatch in unmatched_indices[obj_idx]:
                    if unmatch[0] == neighbor_idx:
                        neighbor_mask = neighbor_masks[unmatch[1]]
                        show_mask(neighbor_mask["segmentation"], axes[neighbor_idx + 1], random_color=False)
                        axes[neighbor_idx + 1].text(neighbor_mask["segmentation"].shape[1] // 2, neighbor_mask["segmentation"].shape[0] // 2, f"N{unmatch[1]}", color=color, fontsize=12, fontweight="bold")

        # Guardar el gráfico
        output_path = os.path.join(output_dir, f"object_Q{obj_idx}_matched_unmatched.png")
        plt.tight_layout()
        plt.savefig(output_path)
        print(f"✅ Gráfico guardado en: {output_path}")
        plt.close(fig)

# --- Llamada a la función ---
output_dir = os.path.join(PLOT_SAVE_ROOT_DIR, "matched_unmatched_plots")
plot_matched_and_unmatched_objects(
    query_image_path=query_image_path,
    neighbor_image_paths=rutas_imagenes_similares,
    query_masks=masks_data_query_image,
    neighbor_masks_list=similar_masks_raw_list,
    matched_indices=all_matched_ref_indices_by_query_obj,
    unmatched_indices=all_closest_unmatched_ref_indices_by_query_obj,
    output_dir=output_dir
)



✅ Gráfico guardado en: /home/imercatoma/FeatUp/plots_single_3/cut_001/matched_unmatched_plots/object_Q0_matched_unmatched.png
✅ Gráfico guardado en: /home/imercatoma/FeatUp/plots_single_3/cut_001/matched_unmatched_plots/object_Q1_matched_unmatched.png
✅ Gráfico guardado en: /home/imercatoma/FeatUp/plots_single_3/cut_001/matched_unmatched_plots/object_Q2_matched_unmatched.png
✅ Gráfico guardado en: /home/imercatoma/FeatUp/plots_single_3/cut_001/matched_unmatched_plots/object_Q3_matched_unmatched.png
✅ Gráfico guardado en: /home/imercatoma/FeatUp/plots_single_3/cut_001/matched_unmatched_plots/object_Q4_matched_unmatched.png
✅ Gráfico guardado en: /home/imercatoma/FeatUp/plots_single_3/cut_001/matched_unmatched_plots/object_Q5_matched_unmatched.png


In [12]:
# %%

## AMM
import torch

@torch.no_grad()
def compute_mahalanobis_map_single(query_fmap, ref_fmaps, regularization=1e-5):
    """
    Calcula el mapa de distancia de Mahalanobis por píxel entre un objeto de consulta
    y k objetos de referencia emparejados.

    Args:
        query_fmap (Tensor): (C, H, W) feature map del objeto de consulta.
        ref_fmaps (List[Tensor]): lista de k tensores (C, H, W) de objetos emparejados.
        regularization (float): término epsilon * I para la inversa numéricamente estable.

    Returns:
        maha_map (Tensor): (H, W) mapa escalar con distancia de Mahalanobis por píxel.
    """
    device = query_fmap.device
    k = len(ref_fmaps)
    C, H, W = query_fmap.shape
    
    if k < 2: # Necesitamos al menos 2 referencias para calcular covarianza
        return torch.zeros(H, W, device=device)

    # Stack: (k, C, H, W)
    ref_stack = torch.stack(ref_fmaps, dim=0)  # (k, C, H, W)

    maha_map = torch.zeros(H, W, device=device)

    for x in range(H):
        for y in range(W):
            vectors = ref_stack[:, :, x, y]         # (k, C)
            mu = vectors.mean(dim=0)                # (C,)
            delta = vectors - mu                    # (k, C)
            cov = delta.T @ delta / (k - 1)         # (C, C) 
            cov += regularization * torch.eye(C, device=device)

            try:
                cov_inv = torch.linalg.inv(cov)     # (C, C)
            except RuntimeError:
                maha_map[x, y] = 0.0
                continue

            v_query = query_fmap[:, x, y]           # (C,)
            diff = (v_query - mu).unsqueeze(0)      # (1, C)

            # Mahalanobis distance squared
            maha_val_squared = (diff @ cov_inv @ diff.T).item()
            # Aplicar la raíz cuadrada para obtener la distancia
            maha_val = torch.sqrt(torch.tensor(maha_val_squared, device=device)).item() # Asegurarse de que sea un tensor para sqrt
            maha_map[x, y] = maha_val

    return maha_map



@torch.no_grad()
def compute_matching_score_map(
    fobj_q,
    all_matched_ref_indices_by_query_obj,
    all_fobj_r_list,
    regularization=1e-5,
    plot_save_dir=None
):
    """
    Calcula los mapas de distancia de Mahalanobis para objetos de consulta
    que tienen referencias emparejadas. Los mapas se normalizan a [0,1].
    """
    matching_maha_maps = []
    all_raw_maha_values = [] 
    print("\n--- Calculando Matching Score Maps (Normalizados) ---")

    for query_idx in range(len(fobj_q)):
        query_fmap = fobj_q[query_idx] # (C, H, W)

        matched_ref_fmaps_list = []
        for neighbor_idx, ref_idx in all_matched_ref_indices_by_query_obj.get(query_idx, []):
            ref_fmap = all_fobj_r_list[neighbor_idx][ref_idx] # (C, H, W)
            matched_ref_fmaps_list.append(ref_fmap)

        if len(matched_ref_fmaps_list) >= 2:
            maha_map = compute_mahalanobis_map_single(
                query_fmap=query_fmap,
                ref_fmaps=matched_ref_fmaps_list,
                regularization=regularization
            )
            
            # Agrega los valores brutos a la lista global para min/max
            all_raw_maha_values.append(maha_map.flatten().cpu()) 
            
            # --- NUEVA Lógica de Normalización ---
            # Normalizar la distancia Mahalanobis a [0, 1].
            # Esto hará que los valores más altos (mayor anomalía) brillen más.
            if maha_map.max() > 1e-8: # Evitar división por cero para mapas planos
                normalized_maha = (maha_map - maha_map.min()) / (maha_map.max() - maha_map.min() + 1e-8)
            else:
                normalized_maha = torch.zeros_like(maha_map) # Si es plano, sigue siendo cero

            print(f"✅ Objeto de consulta {query_idx} emparejado con {len(matched_ref_fmaps_list)} referencias.")
        else:
            # Si no hay suficientes pares coincidentes, el mapa es cero
            normalized_maha = torch.zeros_like(query_fmap[0])
            print(f"ℹ️ Objeto de consulta {query_idx} NO tiene suficientes referencias emparejadas para un Matching Score.")
        
        matching_maha_maps.append(normalized_maha.cpu()) # Agrega el mapa normalizado

       # Visualización (opcional)
        if plot_save_dir:
            plt.figure(figsize=(6, 5))
            plt.imshow(normalized_maha.cpu().numpy(), cmap="hot") # Usa el mapa normalizado
            plt.title(f"Matching Score Map (Normalized) - Obj {query_idx}") # Cambia el título
            plt.axis("off")
            plt.colorbar(label="Normalized Mahalanobis Distance") # Cambia la etiqueta de la barra de color
            plt.tight_layout()
            save_path = os.path.join(plot_save_dir, f"matching_score_obj_{query_idx}.png")
            plt.savefig(save_path)
            plt.close()

    # Calcular el min y max global de TODOS los valores brutos recolectados
    global_min_maha = 0.0
    global_max_maha = 1.0 # Valores por defecto si no hay ningún matched válido

    if all_raw_maha_values:
        combined_raw_values = torch.cat(all_raw_maha_values)
        global_min_maha = combined_raw_values.min().item()
        global_max_maha = combined_raw_values.max().item()
        # Asegurar que el rango no sea cero para evitar división por cero más tarde
        if global_max_maha <= global_min_maha:
            global_max_maha = global_min_maha + 1e-8 

    print(f"Rango global de Mahalanobis RAW para 'Matched': Min={global_min_maha:.4f}, Max={global_max_maha:.4f}")

    return matching_maha_maps, (global_min_maha, global_max_maha) # Devolvemos la tupla con min/max



@torch.no_grad()
def compute_unmatched_score_map(
    fobj_q,
    all_closest_unmatched_ref_indices_by_query_obj,
    all_fobj_r_list,
    regularization=1e-5,
    plot_save_dir=None,
    all_matched_ref_indices_by_query_obj=None,
    # --- RECIBE LA TUPLA CON MIN/MAX ---
    matched_maha_range_global=(0.0, 1.0) 
):
    """
    Calcula los mapas de distancia de Mahalanobis para objetos de consulta
    que NO tienen referencias emparejadas. Los valores se normalizan primero
    usando el rango global de los mapas 'matched', y luego a [0,1] e invierten para mostrar similitud.
    """
    print(f"matched_maha_range_global recibidos: {matched_maha_range_global}")
    
    unmatched_maha_maps = []
    print("\n--- Calculando Unmatched Score Maps (Normalizados a rango Matched e Invertidos para Similitud) ---")

    min_matched_maha_global, max_matched_maha_global = matched_maha_range_global

    # Ya tenemos la validación del rango hecha en compute_matching_score_map,
    # pero podemos añadir una pequeña comprobación aquí por seguridad.
    if max_matched_maha_global <= min_matched_maha_global:
        # Esto debería haber sido ajustado ya, pero por si acaso.
        max_matched_maha_global = min_matched_maha_global + 1e-8 

    for query_idx in range(len(fobj_q)):
        query_fmap = fobj_q[query_idx] 

        matched_refs = all_matched_ref_indices_by_query_obj.get(query_idx, []) if all_matched_ref_indices_by_query_obj else []
        
        if len(matched_refs) >= 2: 
            unmatched_maha_maps.append(torch.zeros_like(query_fmap[0]).cpu())
            print(f"✅ Objeto de consulta {query_idx} ya emparejado. Unmatched map puesto a cero y saltado.")
            continue 

        closest_ref_fmaps = []
        for neighbor_idx, ref_idx in all_closest_unmatched_ref_indices_by_query_obj.get(query_idx, []):
            ref_fmap_closest = all_fobj_r_list[neighbor_idx][ref_idx] 
            closest_ref_fmaps.append(ref_fmap_closest)

        if len(closest_ref_fmaps) >= 2:
            maha_map_raw = compute_mahalanobis_map_single( # Obtén el mapa de Mahalanobis bruto
                query_fmap=query_fmap,
                ref_fmaps=closest_ref_fmaps,
                regularization=regularization
            )
            
            # --- Lógica Clave: Normalización usando el rango global de Matched ---
            # Primero, clip los valores al rango global de los matched.
            # Esto evita que valores extremos en unmatched distorsionen la escala.
            clipped_maha_map = torch.clamp(maha_map_raw, min=min_matched_maha_global, max=max_matched_maha_global)
            print(f"Máximo valor de clipped_maha_map: {clipped_maha_map.max().item()}")
            print(f"Mínimo valor de clipped_maha_map: {clipped_maha_map.min().item()}")
            # Luego, normaliza estos valores clipados al rango [0,1] usando el rango GLOBAL de los matched.
            # Esto alinea la escala de "anomalía" entre matched y unmatched.
            normalized_by_matched_range = (clipped_maha_map - min_matched_maha_global) / (max_matched_maha_global - min_matched_maha_global + 1e-8)
            print(f"Máximo valor de normalized_by_matched_range: {normalized_by_matched_range.max().item()}")
            print(f"Mínimo valor de normalized_by_matched_range: {normalized_by_matched_range.min().item()}")
            # Finalmente, invierte para la visualización de "similitud"
            maha_map_for_display = 1.0 - normalized_by_matched_range 

            print(f"🟡 Objeto de consulta {query_idx} NO emparejado, Mahalanobis (normalizado a rango Matched e invertido) con {len(closest_ref_fmaps)} 'casi-pares'.")
        else:
            maha_map_for_display = torch.zeros_like(query_fmap[0])
            print(f"⚠️ Objeto de consulta {query_idx} no emparejado y sin suficientes 'casi-pares', se pone mapa vacío.")
        
        unmatched_maha_maps.append(maha_map_for_display.cpu()) 

        # Visualización (opcional)
        if plot_save_dir:
            plt.figure(figsize=(6, 5))
            plt.imshow(maha_map_for_display.cpu().numpy(), cmap="hot") 
            plt.title(f"Unmatched Similarity Map (Scaled to Matched Range) - Obj {query_idx}") 
            plt.axis("off")
            plt.colorbar(label="Inverse Normalized Mahalanobis Score (Similarity)") 
            plt.tight_layout()
            save_path = os.path.join(plot_save_dir, f"unmatched_similarity_scaled_to_matched_obj_{query_idx}.png") 
            plt.savefig(save_path)
            plt.close()

    return unmatched_maha_maps



@torch.no_grad()
def build_aggregated_score_map(individual_score_maps_list, final_size=(1024, 1024), title_prefix="Global Score Map", plot_save_dir=None, filename_prefix="global_score_map"):
    """
    Construye un mapa de puntuación agregado (global) a partir de una lista de mapas individuales.

    Args:
        individual_score_maps_list (List[Tensor]): Lista de tensores (H', W') de mapas de puntuación individuales.
                                                   Estos ya deberían estar en CPU.
        final_size (tuple): Tamaño final deseado para el mapa agregado (H, W).
        title_prefix (str): Prefijo para el título del gráfico.
        plot_save_dir (str, optional): Directorio para guardar las visualizaciones.
        filename_prefix (str): Prefijo para el nombre del archivo guardado.

    Returns:
        Tensor: Tensor (H, W) con el mapa de calor total de puntuación.
    """
    H_out, W_out = final_size
    aggregated_score_map = torch.zeros((H_out, W_out), device='cpu')

    print(f"\n--- Construyendo el {title_prefix} ---")
    if not individual_score_maps_list:
        print(f"No hay mapas individuales para {title_prefix}. Retornando un mapa vacío.")
        return aggregated_score_map

    for i, score_map in enumerate(individual_score_maps_list):
        if score_map.dim() == 2:
            score_map_tensor = score_map.unsqueeze(0).unsqueeze(0) # (1,1,H',W')
        else:
            print(f"Advertencia: El mapa de puntuación {i} tiene dimensiones inesperadas: {score_map.shape}. Saltando.")
            continue

        score_resized = F.interpolate(
            score_map_tensor,
            size=(H_out, W_out),
            mode="bilinear",
            align_corners=False
        ).squeeze()

        aggregated_score_map += score_resized

    print(f"Dimensiones del {title_prefix} final: {aggregated_score_map.shape}")

    if plot_save_dir:
        plt.figure(figsize=(8, 7))
        plt.imshow(aggregated_score_map.numpy(), cmap="hot")
        plt.title(title_prefix)
        plt.axis("off")
        plt.colorbar(label="Score Acumulado")
        plt.tight_layout()
        save_path = os.path.join(plot_save_dir, f"{filename_prefix}.png")
        plt.savefig(save_path)
        plt.close()
        print(f"✅ Visualización del {title_prefix} guardada en: {save_path}")

    return aggregated_score_map


def overlay_anomaly_map_on_image(image_rgb_path, anomaly_map, alpha=0.7, cmap='magma', plot_save_dir=None, filename_suffix="overlay"):
    """
    Superpone el mapa de anomalía sobre la imagen original RGB como un heatmap.

    Args:
        image_rgb_path (str): Ruta a la imagen original RGB.
        anomaly_map (Tensor o ndarray): mapa de anomalía (H, W) — debe estar reescalado a 1024×1024.
        alpha (float): transparencia del mapa (0=solo imagen, 1=solo heatmap)
        cmap (str): mapa de color matplotlib a usar ("hot", "jet", etc.)
        plot_save_dir (str, optional): Directorio para guardar la visualización.
        filename_suffix (str): Sufijo para el nombre del archivo guardado.
    """
    print("\n--- Superponiendo el mapa de anomalía final sobre la imagen original ---")
    try:
        image_original_loaded = Image.open(image_rgb_path).convert("RGB")
        image_np = np.array(image_original_loaded)
    except FileNotFoundError:
        print(f"Error: No se encontró la imagen en {image_rgb_path}. No se puede superponer.")
        return

    if isinstance(anomaly_map, torch.Tensor):
        anomaly_np = anomaly_map.cpu().numpy()
    else:
        anomaly_np = anomaly_map

    # Normalizamos el mapa a [0, 1] para visualización
    # Añadimos un pequeño epsilon para evitar división por cero si max == min
    anomaly_min = anomaly_np.min()
    anomaly_max = anomaly_np.max()
    if (anomaly_max - anomaly_min) < 1e-8: # Si todos los valores son iguales
        anomaly_norm = np.zeros_like(anomaly_np)
    else:
        anomaly_norm = (anomaly_np - anomaly_min) / (anomaly_max - anomaly_min)

    # Redimensionamos si las resoluciones no coinciden con la imagen original
    if anomaly_norm.shape[:2] != image_np.shape[:2]:
        anomaly_norm = np.array(Image.fromarray(anomaly_norm).resize(
            (image_np.shape[1], image_np.shape[0]), resample=Image.BILINEAR
        ))

    # Visualización
    plt.figure(figsize=(10, 8))
    plt.imshow(image_np)
    plt.imshow(anomaly_norm, cmap=cmap, alpha=alpha)
    plt.title("Anomaly Heatmap Overlay")
    plt.axis("off")
    plt.tight_layout()

    if plot_save_dir:
        save_path = os.path.join(plot_save_dir, f"{filename_suffix}.png")
        plt.savefig(save_path)
        print(f"✅ Visualización superpuesta guardada en: {save_path}")
    plt.close()




In [13]:
# --- Ejecución del Proceso ---

# --- 1. Calcular Matching Score Maps y obtener el rango global ---
all_matching_score_maps, matched_maha_range_global = compute_matching_score_map(
    fobj_q=fobj_q,
    all_matched_ref_indices_by_query_obj=all_matched_ref_indices_by_query_obj,
    all_fobj_r_list=all_fobj_r_list,
    regularization=1e-5,
    plot_save_dir=PLOT_SAVE_ROOT_DIR
)


--- Calculando Matching Score Maps (Normalizados) ---
✅ Objeto de consulta 0 emparejado con 3 referencias.
✅ Objeto de consulta 1 emparejado con 3 referencias.
ℹ️ Objeto de consulta 2 NO tiene suficientes referencias emparejadas para un Matching Score.
ℹ️ Objeto de consulta 3 NO tiene suficientes referencias emparejadas para un Matching Score.
✅ Objeto de consulta 4 emparejado con 3 referencias.
ℹ️ Objeto de consulta 5 NO tiene suficientes referencias emparejadas para un Matching Score.
Rango global de Mahalanobis RAW para 'Matched': Min=0.0000, Max=6921.1719


In [14]:
# matched_maha_range_global ahora es una tupla (min_value, max_value)

# --- 2. Calcular Unmatched Score Maps (usando el rango global de Matched) ---
all_unmatched_score_maps = compute_unmatched_score_map(
    fobj_q=fobj_q,
    all_closest_unmatched_ref_indices_by_query_obj=all_closest_unmatched_ref_indices_by_query_obj,
    all_fobj_r_list=all_fobj_r_list,
    regularization=1e-5,
    plot_save_dir=PLOT_SAVE_ROOT_DIR,
    all_matched_ref_indices_by_query_obj=all_matched_ref_indices_by_query_obj,
    matched_maha_range_global=matched_maha_range_global # Pasar la tupla aquí
)

matched_maha_range_global recibidos: (0.0, 6921.171875)

--- Calculando Unmatched Score Maps (Normalizados a rango Matched e Invertidos para Similitud) ---
✅ Objeto de consulta 0 ya emparejado. Unmatched map puesto a cero y saltado.
✅ Objeto de consulta 1 ya emparejado. Unmatched map puesto a cero y saltado.
Máximo valor de clipped_maha_map: 6921.171875
Mínimo valor de clipped_maha_map: 0.0
Máximo valor de normalized_by_matched_range: 1.0
Mínimo valor de normalized_by_matched_range: 0.0
🟡 Objeto de consulta 2 NO emparejado, Mahalanobis (normalizado a rango Matched e invertido) con 3 'casi-pares'.
Máximo valor de clipped_maha_map: 6921.171875
Mínimo valor de clipped_maha_map: 0.0
Máximo valor de normalized_by_matched_range: 1.0
Mínimo valor de normalized_by_matched_range: 0.0
🟡 Objeto de consulta 3 NO emparejado, Mahalanobis (normalizado a rango Matched e invertido) con 3 'casi-pares'.
✅ Objeto de consulta 4 ya emparejado. Unmatched map puesto a cero y saltado.
Máximo valor de clipped_m

In [23]:
# 3. Construir el Global Matched Score Map
# Obtener las dimensiones de la imagen original
image_original = Image.open(query_image_path)
H, W = image_original.size

# Construir el Global Matched Score Map
global_matched_score_map = build_aggregated_score_map(
    individual_score_maps_list=all_matching_score_maps,
    final_size=(H, W),
    title_prefix="Global Matched Score Map",
    plot_save_dir=PLOT_SAVE_ROOT_DIR,
    filename_prefix="global_matched_score_map"
)


--- Construyendo el Global Matched Score Map ---
Dimensiones del Global Matched Score Map final: torch.Size([1024, 1024])
✅ Visualización del Global Matched Score Map guardada en: /home/imercatoma/FeatUp/plots_single_3/cut_001/global_matched_score_map.png


In [24]:
# 4. Construir el Global Unmatched Score Map
global_unmatched_score_map = build_aggregated_score_map(
    individual_score_maps_list=all_unmatched_score_maps,
    final_size=(H, W),
    title_prefix="Global Unmatched Score Map",
    plot_save_dir=PLOT_SAVE_ROOT_DIR,
    filename_prefix="global_unmatched_score_map"
)


--- Construyendo el Global Unmatched Score Map ---


Dimensiones del Global Unmatched Score Map final: torch.Size([1024, 1024])
✅ Visualización del Global Unmatched Score Map guardada en: /home/imercatoma/FeatUp/plots_single_3/cut_001/global_unmatched_score_map.png


In [25]:
# 5. Combinar ambos tipos de mapas individuales para el Mapa Global Final (Total)
# Asegúrate de que las listas tengan el mismo número de elementos y en el mismo orden (por query_idx)
combined_individual_score_maps = []
num_queries = len(fobj_q)
for i in range(num_queries):
    # Suma elemento a elemento los mapas de matching y unmatched para cada query
    # Si alguna lista es más corta, se asume que los elementos restantes son cero
    matched_map_for_query = all_matching_score_maps[i] if i < len(all_matching_score_maps) else torch.zeros((H, W), device='cpu')
    unmatched_map_for_query = all_unmatched_score_maps[i] if i < len(all_unmatched_score_maps) else torch.zeros((H, W), device='cpu')
    
    combined_map_for_query_i = matched_map_for_query + unmatched_map_for_query
    combined_individual_score_maps.append(combined_map_for_query_i)


print("\n--- Proceso completado. Revisa la carpeta 'plots_anomalia' para las visualizaciones. ---")



--- Proceso completado. Revisa la carpeta 'plots_anomalia' para las visualizaciones. ---


In [26]:
# 6. Construir el Global Total Anomaly Score Map (el que combina ambos)
global_total_anomaly_score_map = build_aggregated_score_map(
    individual_score_maps_list=combined_individual_score_maps,
    final_size=(1024, 1024),
    title_prefix="Global Total Anomaly Score Map",
    plot_save_dir=PLOT_SAVE_ROOT_DIR,
    filename_prefix="global_total_anomaly_score_map"
)


--- Construyendo el Global Total Anomaly Score Map ---


Dimensiones del Global Total Anomaly Score Map final: torch.Size([1024, 1024])
✅ Visualización del Global Total Anomaly Score Map guardada en: /home/imercatoma/FeatUp/plots_single_3/cut_001/global_total_anomaly_score_map.png


In [27]:
# 7. Superponer los mapas globales en la imagen original
overlay_anomaly_map_on_image(
    image_rgb_path=query_image_path,
    anomaly_map=global_matched_score_map,
    alpha=0.7,
    cmap="magma",
    plot_save_dir=PLOT_SAVE_ROOT_DIR,
    filename_suffix="global_matched_overlay"
)


--- Superponiendo el mapa de anomalía final sobre la imagen original ---
✅ Visualización superpuesta guardada en: /home/imercatoma/FeatUp/plots_single_3/cut_001/global_matched_overlay.png


In [20]:
overlay_anomaly_map_on_image(
    image_rgb_path=query_image_path,
    anomaly_map=global_unmatched_score_map,
    alpha=0.7,
    cmap="magma",
    plot_save_dir=PLOT_SAVE_ROOT_DIR,
    filename_suffix="global_unmatched_overlay"
)

overlay_anomaly_map_on_image(
    image_rgb_path=query_image_path,
    anomaly_map=global_total_anomaly_score_map,
    alpha=0.7,
    cmap="magma",
    plot_save_dir=PLOT_SAVE_ROOT_DIR,
    filename_suffix="global_total_anomaly_overlay"
)

print("\n--- Proceso de generación y visualización de todos los mapas globales completado. ---")
print(f"Revisa la carpeta '{PLOT_SAVE_ROOT_DIR}' para las visualizaciones.")


--- Superponiendo el mapa de anomalía final sobre la imagen original ---
✅ Visualización superpuesta guardada en: /home/imercatoma/FeatUp/plots_single_3/cut_001/global_unmatched_overlay.png

--- Superponiendo el mapa de anomalía final sobre la imagen original ---
✅ Visualización superpuesta guardada en: /home/imercatoma/FeatUp/plots_single_3/cut_001/global_total_anomaly_overlay.png

--- Proceso de generación y visualización de todos los mapas globales completado. ---
Revisa la carpeta '/home/imercatoma/FeatUp/plots_single_3/cut_001' para las visualizaciones.
